# Specify Paths 

In [1]:

fpga_v3_data_collection_path = "/home/ndrakes1/force_estimation/preprocess/fpgav3-data-collection/" # SET THIS ONCE
preproces_path = "/home/ndrakes1/force_estimation/preprocess/force-estimation-fpga-data-preprocess" # SET THIS ONCE

true_preprocess_path = "~/force_estimation/preprocess/"


unit_convert_path = fpga_v3_data_collection_path + "unit_convert/unit_convert.py"
pot_to_encoder_path = preproces_path + "/pot_to_encoder.py"
si_unit_json_path = "/home/ndrakes1/force_estimation/data/XML_FILES/sawRobotIO1394-PSM1-292409.xml.json"
unit_convert_cmd = "python3 " + unit_convert_path + " -c " + si_unit_json_path + " -f " 

train_name_path_csv = "/home/ndrakes1/force_estimation/data/capture_2-17-26/10khz/freespace/train.csv" # SET THIS
val_test_name_path_csv = "/home/ndrakes1/force_estimation/data/capture_2-17-26/10khz/freespace/val.csv" # SET THIS

train_name_path_pot_csv = train_name_path_csv.replace(".csv", "_potEncoder.csv")
val_test_name_path_pot_csv = val_test_name_path_csv.replace(".csv", "_potEncoder.csv")

train_name_path_unit_convert_csv = "/home/ndrakes1/force_estimation/data/capture_2-17-26/10khz/freespace/train_unitConvert.csv" # SET THIS
val_ttest_name_path_unit_convert_csv = "/home/ndrakes1/force_estimation/data/capture_2-17-26/10khz/freespace/val_unitConvert.csv" # SET THIS
train_name_path_unit_convert_pot_csv = train_name_path_pot_csv.replace(".csv", "_unitConvert.csv")
val_ttest_name_path_unit_convert_pot_csv = val_test_name_path_pot_csv.replace(".csv", "_unitConvert.csv")

path_to_data_foleder = "/home/ndrakes1/force_estimation/data/capture_2-17-26/10khz/freespace/"
new_preprocessed_data_folder = "/preprocessed"

ORIGINAL_FREQ = 10000

CUTOFF = True
CUTOFF_SECS = 20

if CUTOFF == True:
    new_preprocessed_data_folder += "_CUTOFF_" + str(CUTOFF_SECS)

INTERPOLATE = True

if INTERPOLATE == True:
    new_preprocessed_data_folder += "_INTERPOLATE" 


FILTER = True
FILTER_FREQ = 30
FILTER_VELOCITY = True
FILTER_POSITION = True

if FILTER == True:
    new_preprocessed_data_folder += "_FILTER_" + str(FILTER_FREQ)
if FILTER_VELOCITY:
    new_preprocessed_data_folder += "_VEL_" 
if FILTER_POSITION:
    new_preprocessed_data_folder += "_POS_" 

DOWNSAMPLE = True
DOWNSAMPLE_FREQ = 60
DOWNSAMPLE_MOVING_AVERAGE = True

if DOWNSAMPLE == True:
    new_preprocessed_data_folder += "_DOWNSAMPLE_" + str(DOWNSAMPLE_FREQ)


USE_POTS = True

if USE_POTS:
    unit_convert_train_input_csv = train_name_path_pot_csv
    unit_convert_val_input_csv = val_test_name_path_pot_csv
    train_preprocess_input_csv = train_name_path_unit_convert_pot_csv
    val_preprocess_input_csv = val_ttest_name_path_unit_convert_pot_csv
    new_preprocessed_data_folder += "POT"
else:
    unit_convert_train_input_csv = train_name_path_csv
    unit_convert_val_input_csv = val_test_name_path_csv
    train_preprocess_input_csv = train_name_path_unit_convert_csv
    val_preprocess_input_csv = val_ttest_name_path_unit_convert_csv

unit_convert_train_cmd = unit_convert_cmd + unit_convert_train_input_csv
unit_convert_val_test_cmd = unit_convert_cmd + unit_convert_val_input_csv


In [2]:
## making preprocessed data dir 
import subprocess

new_dir = path_to_data_foleder + new_preprocessed_data_folder
train_sub = new_dir + "/train/joints/"
val_sub = new_dir + "/val/joints/"
test_sub = new_dir + "/test/joints/"
mk_new_dir_cmd = "mkdir -p " + train_sub + " " + val_sub + " " + test_sub

subprocess.run(mk_new_dir_cmd, shell=True)

CompletedProcess(args='mkdir -p /home/ndrakes1/force_estimation/data/capture_2-17-26/10khz/freespace//preprocessed_CUTOFF_20_INTERPOLATE_FILTER_30_VEL__POS__DOWNSAMPLE_60POT/train/joints/ /home/ndrakes1/force_estimation/data/capture_2-17-26/10khz/freespace//preprocessed_CUTOFF_20_INTERPOLATE_FILTER_30_VEL__POS__DOWNSAMPLE_60POT/val/joints/ /home/ndrakes1/force_estimation/data/capture_2-17-26/10khz/freespace//preprocessed_CUTOFF_20_INTERPOLATE_FILTER_30_VEL__POS__DOWNSAMPLE_60POT/test/joints/', returncode=0)

## paths for high freq preprocessing

In [3]:

preprocessing_script_path = preproces_path + "/preprocessing.py"
split_val_test_path = preproces_path + "/split_val_test.py"

preprocessing_train_cmd = "python3 " + preprocessing_script_path + " " + train_preprocess_input_csv + " " + train_sub + "interpolated_all_joints.csv"
preprocessing_val_test_cmd ="python3 " + preprocessing_script_path + " " + val_preprocess_input_csv + " " + val_sub + "interpolated_all_joints.csv"

print(preprocessing_train_cmd)

python3 /home/ndrakes1/force_estimation/preprocess/force-estimation-fpga-data-preprocess/preprocessing.py /home/ndrakes1/force_estimation/data/capture_2-17-26/10khz/freespace/train_potEncoder_unitConvert.csv /home/ndrakes1/force_estimation/data/capture_2-17-26/10khz/freespace//preprocessed_CUTOFF_20_INTERPOLATE_FILTER_30_VEL__POS__DOWNSAMPLE_60POT/train/joints/interpolated_all_joints.csv


# Unit Conversion to SI units

In [4]:

if USE_POTS:
    pot_convert_train_cmd = "python3 " + pot_to_encoder_path + " " + train_name_path_csv + " " + train_name_path_pot_csv
    pot_convert_val_test_cmd = "python3 " + pot_to_encoder_path + " " + val_test_name_path_csv + " " + val_test_name_path_pot_csv
    subprocess.run(pot_convert_val_test_cmd, shell=True)
    subprocess.run(pot_convert_train_cmd, shell=True)

subprocess.run(unit_convert_val_test_cmd, shell=True)
subprocess.run(unit_convert_train_cmd, shell=True)

Data written to /home/ndrakes1/force_estimation/data/capture_2-17-26/10khz/freespace/val_potEncoder_unitConvert.csv
Data written to /home/ndrakes1/force_estimation/data/capture_2-17-26/10khz/freespace/train_potEncoder_unitConvert.csv


CompletedProcess(args='python3 /home/ndrakes1/force_estimation/preprocess/fpgav3-data-collection/unit_convert/unit_convert.py -c /home/ndrakes1/force_estimation/data/XML_FILES/sawRobotIO1394-PSM1-292409.xml.json -f /home/ndrakes1/force_estimation/data/capture_2-17-26/10khz/freespace/train_potEncoder.csv', returncode=0)

# High Freq Preprocessing

In [5]:
# # interpolate train and place it in correct directory
# # train
subprocess.run(preprocessing_train_cmd, shell=True)
print("finished preprocessing train ")

## val test
subprocess.run(preprocessing_val_test_cmd, shell=True)
print("preprocess split and val")

# # split
split_val_test_cmd = "python3 " + preproces_path + "/split_val_test.py " + " " + val_sub + "interpolated_all_joints.csv"
subprocess.run(split_val_test_cmd, shell=True)
print("split into val and test")

!mv {preproces_path}/val.csv {val_sub}interpolated_all_joints.csv
!mv {preproces_path}/test.csv {test_sub}interpolated_all_joints.csv


finished preprocessing train 
preprocess split and val
Validation set saved to val.csv 996800 rows
Test set saved to test.csv 996800 rows
split into val and test


## Preprocessed Training File Paths

In [6]:
import os
import pandas as pd

train_file = os.path.join(train_sub, "interpolated_all_joints.csv")
val_file = os.path.join(val_sub, "interpolated_all_joints.csv")
test_file = os.path.join(test_sub, "interpolated_all_joints.csv")

train_df = pd.read_csv(train_file, header=None)
val_df =  pd.read_csv(val_file, header=None)
test_df = pd.read_csv(test_file, header=None)

## Cutoff


In [7]:
from cutoff import truncate_dataframe


if (CUTOFF == True):
    ## TRAIN
    FREQ =  ORIGINAL_FREQ if DOWNSAMPLE == False else DOWNSAMPLE_FREQ
    ## TRAIN
    train_df = truncate_dataframe(train_df, CUTOFF_SECS, FREQ)
    val_df = truncate_dataframe(val_df, CUTOFF_SECS, FREQ)
    test_df = truncate_dataframe(test_df, CUTOFF_SECS, FREQ)

total rows:  5853072
total rows:  996800
total rows:  996800


## FIR Filtering

In [8]:
import importlib
import filter  # regular import first
importlib.reload(filter)



if (FILTER == True):
    # Design filter
    fir_coeffs = filter.design_fir_filter(filter_type='kaiser', fs=ORIGINAL_FREQ, fC=FILTER_FREQ, order=30)

    # Apply to train/val/test CSVs
    train_df = filter.apply_filter_to_torque_feedback_df(train_df, fir_coeffs, filter_velocity=FILTER_VELOCITY, filter_position=FILTER_POSITION)
    val_df = filter.apply_filter_to_torque_feedback_df(val_df, fir_coeffs, filter_velocity=FILTER_VELOCITY, filter_position=FILTER_POSITION)
    test_df = filter.apply_filter_to_torque_feedback_df(test_df, fir_coeffs, filter_velocity=FILTER_VELOCITY, filter_position=FILTER_POSITION)

    print("FILTERED")

FILTERED


## Downsample


In [9]:
from downsample import downsample_dataframe

if (DOWNSAMPLE == True):
    train_df = downsample_dataframe(train_df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)
    val_df = downsample_dataframe(val_df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)
    test_df = downsample_dataframe(test_df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)


# Interpolate

In [10]:
from interpolate_timestamps import interpolate_dataframe_to_sample_rate

if (INTERPOLATE == True):

    freq = 0

    if (DOWNSAMPLE):
        freq = DOWNSAMPLE_FREQ
    else:   
        freq = ORIGINAL_FREQ
    
    train_df = interpolate_dataframe_to_sample_rate(train_df, freq)
    val_df = interpolate_dataframe_to_sample_rate(val_df, freq)
    test_df = interpolate_dataframe_to_sample_rate(test_df, freq)

[0.00000000e+00 1.47003817e-02 3.12999001e-02 ... 5.85031900e+02
 5.85048500e+02 5.85058700e+02]
[0.00000000e+00 1.46999880e-02 3.13000419e-02 ... 9.93988998e+01
 9.94155000e+01 9.94256999e+01]
[0.00000000e+00 1.46999897e-02 3.12999355e-02 ... 9.93989024e+01
 9.94155000e+01 9.94256999e+01]


## SAVE TO CSV

In [11]:
train_df.to_csv(train_file, index=False, header=False)
val_df.to_csv(val_file, index=False, header=False)
test_df.to_csv(test_file, index=False, header=False)